In [1]:
# Import libraries and modules
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import keras
import string
import nltk
from autocorrect import Speller
import re
import datetime, os


In [2]:
data = pd.read_csv('datasets/dataset_complete_balanced.csv', encoding='utf8')
print(len(data))
data.head()

19107


,emotion,text
0,anger,he looks offended i tell perceived man man anger
1,anger,we school room audiovisual classes we watchin...
2,anger,oh jim i told no pets it make mess house
3,anger,getting terrible meals paying heaps
4,anger,when drinking glass water i found cockroach wa...


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

data = data.sample(frac=1,random_state=42)
data = data[['text','emotion']]
data.emotion = LabelEncoder().fit_transform(data.emotion)
data.head()

,text,emotion
11632,you late,6
9380,hey mary,4
4518,the house fire i saved dog i afraid fire verg...,2
6388,making sexual pass cousin i close personal re...,3
1443,us banks also lost heavily furious bundesbank ...,0


In [4]:
test_dataframe = data.sample(frac=0.2, random_state=1337)
train_dataframe = data.drop(test_dataframe.index)

val_dataframe = train_dataframe.sample(frac=0.25, random_state=1337)
train_dataframe = train_dataframe.drop(val_dataframe.index)


print("Using %d samples for training and %d for validation and %d for test" % (len(train_dataframe), len(val_dataframe),len(test_dataframe)))

Using 11464 samples for training and 3822 for validation and 3821 for test


In [5]:
def dataframe_to_dataset(dataframe):
    data_copy = data.copy()
    labels = data_copy.pop("emotion")
    ds = tf.data.Dataset.from_tensor_slices((dict(data_copy), labels))
    ds = ds.shuffle(buffer_size=len(data_copy))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)
test_ds = dataframe_to_dataset(test_dataframe)

In [6]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'text': <tf.Tensor: shape=(), dtype=string, numpy=b'i felt joy i saw daughter separation twenty  days '>}
Target: tf.Tensor(5, shape=(), dtype=int32)


In [7]:
train_ds = train_ds.shuffle(10000).batch(1024)
val_ds = val_ds.shuffle(10000).batch(1024)
test_ds = test_ds.shuffle(10000).batch(1024)

In [8]:

# Create embedding layer
#hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string)
model_new = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1", dtype=tf.string, input_shape=[], output_shape=[128]),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model_new.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2

In [9]:
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


In [10]:
from keras import callbacks


# Exponantial Learning Rate Scheduler function
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr*tf.math.exp(-0.1)

# Define early stopping to prevent overfit
earlystopping = callbacks.EarlyStopping(monitor='val_accuracy',
                                        mode='max',
                                        patience=10,
                                        restore_best_weights=True,
                                        verbose=1)

# Define lr scheduler to prevent overfit
lr_scheduler = callbacks.LearningRateScheduler(scheduler, verbose=1)

In [11]:
model_new.compile(loss = 'sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.006),
              metrics=['accuracy'])

history = model_new.fit(train_ds, epochs=100, validation_data=val_ds, callbacks = [tensorboard_cb])

%tensorboard --logdir logs

Epoch 1/100
19/19 [==============================] - 10s 405ms/step - loss: 2.2246 - accuracy: 0.1582 - val_loss: 1.9604 - val_accuracy: 0.2723
Epoch 2/100
19/19 [==============================] - 1s 68ms/step - loss: 1.9205 - accuracy: 0.2777 - val_loss: 1.7660 - val_accuracy: 0.3332
Epoch 3/100
19/19 [==============================] - 1s 50ms/step - loss: 1.7291 - accuracy: 0.3399 - val_loss: 1.6047 - val_accuracy: 0.3791
Epoch 4/100
19/19 [==============================] - 1s 48ms/step - loss: 1.6017 - accuracy: 0.3921 - val_loss: 1.5117 - val_accuracy: 0.4336
Epoch 5/100
19/19 [==============================] - 1s 76ms/step - loss: 1.5016 - accuracy: 0.4355 - val_loss: 1.4231 - val_accuracy: 0.4689
Epoch 6/100
19/19 [==============================] - 1s 38ms/step - loss: 1.4213 - accuracy: 0.4662 - val_loss: 1.3568 - val_accuracy: 0.4973
Epoch 7/100
19/19 [==============================] - 1s 37ms/step - loss: 1.3626 - accuracy: 0.5006 - val_loss: 1.3024 - val_accuracy: 0.5152
Epoc

Reusing TensorBoard on port 6006 (pid 17832), started 20:26:48 ago. (Use '!kill 17832' to kill it.)

In [14]:
# Save the model
model_new.save('emotion_analysis')

INFO:tensorflow:Assets written to: emotion_analysis\assets
INFO:tensorflow:Assets written to: emotion_analysis\assets


In [13]:
results = model_new.evaluate(test_ds)


19/19 [==============================] - 0s 21ms/step - loss: 0.0950 - accuracy: 0.9644
